In [1]:
%reload_ext autoreload
%autoreload 2

# Load the data

In [2]:
from result_saver import SaverProvider
provider = SaverProvider()

In [69]:
from Scratch import metadata_loader
import numpy as np
from tqdm import tqdm
from time import sleep

DEVICE = "ibm_torino"
LOGICAL = str(1)
XBASIS = False
ROUNDS = 20

state = "X" if XBASIS else "Z"
state += LOGICAL
print(f"state = {state}")


# Load the metadata
while True:
    try:
        md = metadata_loader(True, True)
        break
    except:
        sleep(5)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["code"] == "RepetitionCodeCircuit"]
md = md[md["descr"] == 'subset RepCodes']
md = md.dropna(subset=["rounds"])
md = md[md["meas_level"] == 1]
md['rounds'] = md['rounds'].astype(int)
md['distance'] = md['distance'].astype(int)

md = md[md["backend_name"] == DEVICE]
md = md[md["logical"] == LOGICAL]
md = md[md["xbasis"] == XBASIS]
md = md[md["rounds"] == ROUNDS]

md = md[0:1]
print("shape:", md.shape)
# md[18:]
md

state = Z1
shape: (1, 18)


,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,num_qubits,job_status,execution_date,code,distance,rounds,logical,descr,resets,xbasis,path_info
4498,2024-03-23 10:55:31.224000+01:00,Torino_RepCodes_jobs,ibm_torino,cqzae4rktf3g00883ew0,"[Subset 56, 20 rounds, 1 log, xbasis=False]",1.0,3303.0,NaN,JobStatus.DONE,2024-03-23 11:35:10.771175+01:00,RepetitionCodeCircuit,56,20,1,subset RepCodes,NaN,False,"{'mean_gate_error': 0.006839442881539001, 'min..."


# Take single job

In [70]:
job_id = md["job_id"].values[0]
d = md["distance"].values[0]
T = md["rounds"].values[0]

job = provider.retrieve_job(job_id)
memory = job.result().get_memory()

print(f"nb of shots: {len(memory)}")

nb of shots: 3303


# Get needed decoding objects

In [71]:
from soft_info import load_calibration_memory
from src import cpp_soft_info as csi

lin = [0.6, 1.2, 1]
num_points = 20
bandwidths = np.linspace(lin[0], lin[1], lin[2])
rel_error = 1

all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_job=job_id, 
                                             double_msmt=False, post_process=True)
kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


all_memories_PS, _, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_job=job_id, 
                                                         double_msmt=True, post_process=True)
kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


Specified job execution date: 2024-03-23 11:35:10.771175+01:00
Found jobs for backend ibm_torino with closest execution date 2024-03-23 10:35:40.641635+00:00.
Specified job execution date: 2024-03-23 11:35:10.771175+01:00
Found jobs for backend ibm_torino with closest execution date 2024-03-23 10:36:07.872547+00:00.


In [72]:
print(all_memories[0].keys())
print(all_memories_PS[0].keys())

dict_keys(['mmr_0', 'mmr_1'])
dict_keys(['mmr_0', 'mmr_1', 'mmr_0_scnd', 'mmr_1_scnd'])


In [73]:
# Get the msmt errors
p_soft_mean = 0
p_hard_mean = 0
for key, value in msmt_err_dict_PS.items():
    p_soft_mean += value['p_soft']
    p_hard_mean += value['p_hard']
p_soft_mean /= len(msmt_err_dict_PS)
p_hard_mean /= len(msmt_err_dict_PS)

print(f"p_soft_mean = {p_soft_mean}")
print(f"p_hard_mean = {p_hard_mean}")

p_soft_mean = 0.008314078406097044
p_hard_mean = 0.02450996814374167


In [74]:
from soft_info import inv_qubit_mapping, get_repcode_IQ_map

layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
link_qubits = list(layout_des['link_qubit'].values())
code_qubits = list(layout_des['code_qubit'].values())

big_layout = link_qubits + code_qubits
inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))


# Get countMat and pSoft

In [75]:
from datetime import datetime
from soft_info import gaussianIQConvertor

HANDLE_OUTLIERS = True

print(f"Starting to get pSoft and countMat at {datetime.now()}")
pSoft, countMat = csi.iqConvertor(memory, inverted_q_map, kde_dict, rel_error, -1,
                                            handleOutliers = HANDLE_OUTLIERS)

print(f"Starting to get pSoft_PS and countMat_PS at {datetime.now()}")
pSoft_PS, countMat_PS = csi.iqConvertor(memory, inverted_q_map, kde_dict_PS, rel_error, -1,
                                                handleOutliers = HANDLE_OUTLIERS)

# print(f"Starting to get pSoftG at {datetime.now()}")
# countMatG, pSoftG = gaussianIQConvertor(memory, inverted_q_map, gmm_dict)

Starting to get pSoft and countMat at 2024-04-09 19:19:37.271853
Starting to get pSoft_PS and countMat_PS at 2024-04-09 19:20:10.248013


# Get the noise model 

In [76]:
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict


noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = md["execution_date"].values[0])
avgs = get_avgs_from_dict(noise_dict, big_layout)
noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
readout = avgs["readout"]
noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]

print(f"noise_list = {noise_list}")
print(f"readout = {readout}")

19:20:33 Warning: Z0 decoding. Negative T2 error -0.06 % for qubit 90, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
19:20:36 Warning: Could not get two gate error of ECR due to 'Could not find the desired property for ecr', taking CX instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 67
19:20:36 Warning: Could not get two gate error of CX due to 'Could not find the desired property for cx', taking 0.5 instead.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 71
noise_list = [0.006824523916809919, 0.0005384309815914904, 0.0038028275581356076, 0.0050524243666174445, 0.03282404654983871, 0.02450996814374167, 0.008314078406097044]
readout = 0.027490990990990994


In [77]:
# Stim codes
from soft_info import RepetitionCodeStimCircuit

subsampling = False
_RESETS = True

noise_list_new = noise_list.copy()
# noise_list_new = [err*10 for err in noise_list_new]

code_mean_for_soft = RepetitionCodeStimCircuit(d, T, XBASIS, _RESETS, noise_list=noise_list_new,
                                                subsampling=subsampling, no_fin_soft=True, layout=None,
                                                msmt_err_dict=None)
model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

code_mean_for_hard = RepetitionCodeStimCircuit(d, T, XBASIS, _RESETS, noise_list=noise_list_new,
                                                subsampling=subsampling, no_fin_soft=False, layout=None,
                                                msmt_err_dict=None)
model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()

# Decoding

In [78]:
_RESETS_new = False

offset = 0.005
pSoft_new = pSoft + offset
pSoft_PS_new = pSoft_PS + offset

res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat, pSoft, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat, pSoft, T,
                                      int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_PS = csi.decodeConvertorAll(model_mean_for_soft, countMat_PS, pSoft_PS, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=False)
res_h_K_mean_PS = csi.decodeConvertorAll(model_mean_for_hard, countMat_PS, pSoft_PS, T,
                                        int(LOGICAL), _RESETS_new, decode_hard=True)

res_s_K_mean_new = csi.decodeConvertorAll(model_mean_for_soft, countMat, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)
res_s_K_mean_PS_new = csi.decodeConvertorAll(model_mean_for_hard, countMat, pSoft_new, T,
                                            int(LOGICAL), _RESETS_new, decode_hard=False)

In [79]:
print(f"d: {d}, T: {T}, SHOTS: {len(pSoft)}, LOGICAL: {LOGICAL}, XBASIS: {XBASIS}, RESETS: {_RESETS}, DOUBLE_MSMTS: {True}")
print(f"noise_list_new: {noise_list_new}\n")

print(f"soft KDE: {res_s_K_mean.num_errors} out of {len(pSoft)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"soft KDE_new: {res_s_K_mean_new.num_errors} out of {len(pSoft)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"hard KDE: {res_h_K_mean.num_errors} out of {len(pSoft)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")

print() 

print(f"soft KDE_PS: {res_s_K_mean_PS.num_errors} out of {len(pSoft_PS)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"soft KDE_new_PS: {res_s_K_mean_PS_new.num_errors} out of {len(pSoft_PS)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"hard KDE_PS: {res_h_K_mean_PS.num_errors} out of {len(pSoft_PS)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")


d: 56, T: 20, SHOTS: 3303, LOGICAL: 1, XBASIS: False, RESETS: True, DOUBLE_MSMTS: True
noise_list_new: [0.006824523916809919, 0.0005384309815914904, 0.0038028275581356076, 0.0050524243666174445, 0.03282404654983871, 0.02450996814374167, 0.008314078406097044]

soft KDE: 0 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]
soft KDE_new: 2 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]
hard KDE: 3 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]

soft KDE_PS: 1 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]
soft KDE_new_PS: 3 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]
hard KDE_PS: 3 out of 3303 shots, handleOutliers: True, bandwidth: [0.6]


In [80]:
print(f"p_soft_mean: {p_soft_mean}, p_hard_mean: {p_hard_mean}")
print(f"mean pSoft: {np.mean(pSoft)}, mean pSoft_PS: {np.mean(pSoft_PS)}")


print(f"mean pSoft_new: {np.mean(pSoft_new)}, mean pSoft_PS_new: {np.mean(pSoft_PS_new)}")

p_soft_mean: 0.008314078406097044, p_hard_mean: 0.02450996814374167
mean pSoft: 0.06042932388898831, mean pSoft_PS: 0.02489950177745181
mean pSoft_new: 0.06542932388898831, mean pSoft_PS_new: 0.029899501777451796
